# LDA and PyLDAvis

- Identify topics and cluster articles using topics or remove unrelated topics

In [ ]:
import time
import pandas as pd
import numpy as np
# import nltk
import gensim
from gensim import models
from gensim import corpora
import matplotlib.pyplot as plt
%matplotlib inline
!pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
import texthero as hero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# Preprocessing

In [1]:
custom_pipeline = [hero.preprocessing.drop_no_content,
                   hero.preprocessing.remove_urls,
                   hero.preprocessing.remove_diacritics,
                  hero.preprocessing.remove_digits,
                   hero.preprocessing.remove_punctuation,
                   hero.preprocessing.lowercase,
                   hero.preprocessing.remove_stopwords,
                   hero.preprocessing.remove_whitespace]

NameError: ignored

In [ ]:
dfc_nc = pd.read_csv('/drive/My Drive/data/no_chicago.csv')

In [ ]:
lda_df = dfc_nc[['title','text']]
lda_df.shape

In [ ]:
%%time 
lda_df['clean_text'] = hero.clean(lda_df['text'], custom_pipeline)
# lda_df = lda_df[['clean_text']]

In [ ]:
lda_df['leng'] = lda_df['clean_text'].apply(lambda x: len(str(x)))

In [ ]:
lda_df = lda_df[lda_df['clean_text'].ne('')] #ne – not equal

In [ ]:
lda_df_new = lda_df[lda_df.leng > 250]
lda_df_new['leng'].describe()

# LDA

In [ ]:
text_list = lda_df_new['clean_text'].str.split().tolist()
text_list[0:1]

In [ ]:
from gensim.models.phrases import Phrases, Phraser
#bigrams
bigram = Phrases(text_list, min_count=5, threshold=2, max_vocab_size = 500000)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

data_bigrams = make_bigrams(text_list)

In [ ]:
len(data_bigrams),len(text_list)

In [ ]:
text_list[0]

In [ ]:
data_bigrams[0]

In [ ]:
from collections import defaultdict
# remove words that appear only once
frequency = defaultdict(int)
for text in text_list:
    for token in text:
        frequency[token] += 1

doc_token_cleaned_list = [[token for token in text if frequency[token] > 1]for text in text_list]

# doc_token_cleaned_list[0:2] #remove few words from list

In [ ]:
len(doc_token_cleaned_list)

In [ ]:
tokens_combo = doc_token_cleaned_list + data_bigrams
len(tokens_combo)

In [ ]:
from gensim.corpora.dictionary import Dictionary

dictionary = corpora.Dictionary(tokens_combo)
dictionary.filter_extremes(no_below=3, no_above=0.8, keep_n=80000)
dictionary.compactify()
# print(dictionary)

In [ ]:
doc_term_matrix  = [dictionary.doc2bow(doc) for doc in tokens_combo]  #doc2bow or corpus is indexed&freqdist of words per documents
# doc_term_matrix

# LDA - Estimation

In [ ]:
%%time
from gensim.models import LdaMulticore
lda_model = LdaMulticore(corpus=doc_term_matrix,id2word=dictionary, num_topics=10, passes=20)

In [ ]:
lda_model.print_topics()

[(0,
  '0.006*"get" + 0.006*"may" + 0.006*"one" + 0.006*"also" + 0.005*"use" + 0.004*"time" + 0.004*"need" + 0.004*"well" + 0.003*"information" + 0.003*"money"'),
 (1,
  '0.045*"tool_works" + 0.013*"company_stock" + 0.009*"last_quarter" + 0.008*"additional_shares" + 0.007*"research_report" + 0.007*"owns_shares" + 0.007*"third_quarter" + 0.006*"3rd_quarter" + 0.006*"products_company" + 0.006*"get_rating"'),
 (2,
  '0.007*"market" + 0.006*"new" + 0.005*"company" + 0.005*"business" + 0.003*"com" + 0.003*"million" + 0.003*"information" + 0.003*"services" + 0.003*"including" + 0.003*"industry"'),
 (3,
  '0.008*"one" + 0.006*"like" + 0.006*"said" + 0.005*"also" + 0.005*"people" + 0.005*"time" + 0.004*"get" + 0.004*"says" + 0.004*"see" + 0.004*"many"'),
 (4,
  '0.070*"shares" + 0.054*"company" + 0.050*"stock" + 0.032*"rating" + 0.022*"quarter" + 0.019*"year" + 0.014*"average" + 0.014*"billion" + 0.012*"holdings" + 0.012*"inc"'),
 (5,
  '0.016*"covid" + 0.012*"health" + 0.009*"state" + 0.009*"

In [ ]:
lda_model.save('/drive/My Drive/data/lda_nochicago1.model')

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [ ]:
lda_display = gensimvis.prepare(lda_model, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
